# Section 8, Lecture 3: Advanced LangGraph - Enhance AI Workflows with Memory and BranchingWelcome to Lecture 3 of Section 8! Today, you’ll take your LangGraph skills to the next level by adding memory and branching logic to your customer support triage system.### What You’ll Build- A LangGraph workflow with memory to track past customer inquiries.- Branching logic to handle different inquiry types and personalize responses.This will make your AI workflow more intelligent, reliable, and business-ready.

In [ ]:
# Step 1: Install LangGraph# Run this in your terminal if not already installed:# uv pip install langgraphtry:    import langgraph    print("LangGraph is installed! Version:", langgraph.__version__)except ImportError:    print("LangGraph not found. Please install it using the command above.")

## Step 1: Installing LangGraphThis cell checks if LangGraph is installed. If not, install it using `uv pip install langgraph`.We're using `uv` as our package manager from Section 2 for speed and reliability.

In [ ]:
# Step 2: Load your OpenAI API keyfrom dotenv import load_dotenvimport osload_dotenv()openai_api_key = os.getenv("OPENAI_API_KEY")if openai_api_key:    print("API key loaded successfully!")else:    print("API key not found. Check your .env file.")

## Step 2: Load OpenAI API KeyWe use your `.env` file to securely load your GPT-4 key. Make sure `OPENAI_API_KEY=your-key-here` exists.This key powers the GPT-4 responses in your workflow.

In [ ]:
# Step 3: Set up LangGraph workflow with memoryfrom langgraph.graph import StateGraph, ENDfrom langchain_openai import ChatOpenAIfrom typing import Dict, Anyllm = ChatOpenAI(model="gpt-4", api_key=openai_api_key)class WorkflowState(Dict[str, Any]):    passworkflow = StateGraph(WorkflowState)print("Workflow with memory set up successfully!")

## Step 3: Set Up LangGraph Workflow with MemoryHere we extend your LangGraph workflow from Lecture 2 by adding memory. This enables the system to track previous inquiries, personalize responses, and improve context-aware triage.

In [ ]:
# Step 4: Define nodes with branching logicdef classify(state):    inquiry = state.get("inquiry")    prompt = f"Classify this inquiry: {inquiry} (question, complaint, follow-up, urgent)"    response = llm.invoke(prompt).content    state["inquiry_type"] = response.strip().lower()    return statedef check_memory(state):    past = state.get("history", [])    inquiry = state.get("inquiry")    if any(inquiry in p for p in past):        state["inquiry_type"] = "follow-up"    return statedef route(state):    itype = state.get("inquiry_type")    if "urgent" in itype or "follow-up" in itype:        state["route"] = "priority"    else:        state["route"] = "standard"    return statedef respond(state):    route = state.get("route")    inquiry = state.get("inquiry")    if route == "priority":        state["response"] = "Escalated for human review."    else:        prompt = f"Respond to this inquiry: {inquiry}"        state["response"] = llm.invoke(prompt).content    state.setdefault("history", []).append(inquiry)    return state# Add nodesworkflow.add_node("classify", classify)workflow.add_node("check_memory", check_memory)workflow.add_node("route", route)workflow.add_node("respond", respond)# Define workflowworkflow.set_entry_point("classify")workflow.add_edge("classify", "check_memory")workflow.add_edge("check_memory", "route")workflow.add_edge("route", "respond")workflow.add_edge("respond", END)app = workflow.compile()print("Nodes and edges with branching defined successfully!")

## Step 4: Define Nodes with Branching Logic- `classify`: Determines if the inquiry is a question, complaint, follow-up, or urgent.- `check_memory`: Looks into the inquiry history to detect follow-ups.- `route`: Decides if the inquiry should go to priority or standard queue.- `respond`: Either drafts an automated reply or escalates for human review.Edges are defined to build a smart, conditional flow with branching based on inquiry type and memory.

In [ ]:
# Step 5: Run the enhanced workflowinitial_state = WorkflowState(    inquiry="My order #12345 is delayed again—what’s going on?",    history=["Order #12345 was delayed last week."],    inquiry_type="",    route="",    response="")result = app.invoke(initial_state)print("Final Output:\n", result.get("response"))

## Step 5: Run the Enhanced WorkflowWe run the enhanced system with a follow-up inquiry. The system:1. Classifies it as a follow-up or urgent.2. Checks memory and confirms previous inquiry.3. Routes it to the priority queue.4. Escalates the case for human review.This shows LangGraph’s ability to personalize workflows using memory and handle logic with reliability.

## Wrap-UpYou just built an advanced LangGraph workflow with:- **Memory** to track past inquiries- **Branching** to personalize the logic flow- **Reliable triage automation** ready for scale**Next up:** The capstone project! You’ll combine all your skills to build a complete LangGraph system tailored to your business needs.**Challenge:** Try changing the inquiry or adding more memory scenarios. What happens if you simulate a brand new inquiry vs. a recurring one?Drop your outputs in the Q&A!